In [16]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns 

In [17]:
file_path = '../Data/DM2425_ABCDEats_DATASET.csv'
df = pd.read_csv(file_path)
shape_of_df = df.shape
shape_of_df

(31888, 56)

In [18]:
df.head()


,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,...,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23
0,1b8f824d5e,2360,18.0,2,5,1,0.0,1,DELIVERY,DIGI,...,0,0,0,0,2,0,0,0,0,0
1,5d272b9dcb,8670,17.0,2,2,2,0.0,1,DISCOUNT,DIGI,...,0,0,0,0,0,0,0,0,0,0
2,f6d1b2ba63,4660,38.0,1,2,2,0.0,1,DISCOUNT,CASH,...,0,0,0,0,0,0,0,0,0,0
3,180c632ed8,4660,NaN,2,3,1,0.0,2,DELIVERY,DIGI,...,1,0,0,0,0,0,0,0,0,0
4,4eb37a6705,4660,20.0,2,5,0,0.0,2,-,DIGI,...,0,0,0,0,0,0,0,0,0,0


## remove duplicates

In [19]:
df['customer_id'].unique().shape

(31875,)

oh no! there are repetitions!

In [20]:
#find repeated customer_id
repetitions = df['customer_id'].value_counts()
#keep only the customer_id that are repeated
repetitions = repetitions[repetitions > 1]
repetitions = repetitions.index
repetitions

Index(['742ca068fc', 'b55012ee1c', 'df91183978', '6bbf5f74cd', '24251eb7da',
       '201a13a34d', 'b8e7a643a4', 'cc08ef25ce', '8aa9bbc147', '671bf0c738',
       '06018a56be', 'fac7984c0d', 'cf563a0a98'],
      dtype='object', name='customer_id')

In [21]:
for customer_id in repetitions:
    #check if the rows are equal
    if not df[df['customer_id'] == customer_id].equals(df[df['customer_id'] == customer_id]):
        print('Not equal')
    else:
        print('Equal')  

df = df.drop_duplicates(subset='customer_id', keep='first')


Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal
Equal


## non numerical columns

In [22]:
non_numerical_columns = df.select_dtypes(include=['object']).columns.tolist()
print(non_numerical_columns)

['customer_id', 'customer_region', 'last_promo', 'payment_method']


In [23]:
for col in non_numerical_columns:
    print(f'{col}: {df[col].unique()}')

customer_id: ['1b8f824d5e' '5d272b9dcb' 'f6d1b2ba63' ... 'f74ad8ce3f' 'f7b19c0241'
 'fd40d3b0e0']
customer_region: ['2360' '8670' '4660' '2440' '-' '4140' '2490' '8370' '8550']
last_promo: ['DELIVERY' 'DISCOUNT' '-' 'FREEBIE']
payment_method: ['DIGI' 'CASH' 'CARD']


In [24]:
df.loc[df['customer_region'] == '-', 'customer_region'] = np.nan

In [25]:
df['customer_region'].unique()  

array(['2360', '8670', '4660', '2440', nan, '4140', '2490', '8370',
       '8550'], dtype=object)

In [26]:
df['last_promo'].unique()

array(['DELIVERY', 'DISCOUNT', '-', 'FREEBIE'], dtype=object)

In [27]:
df.loc[df['last_promo'] == '-', 'last_promo'] = "No_Promo"
df['last_promo'].unique()

array(['DELIVERY', 'DISCOUNT', 'No_Promo', 'FREEBIE'], dtype=object)

In [31]:
promo_count = df['last_promo'].value_counts()
promopercentage = promo_count / promo_count.sum() * 100
print(promopercentage)

last_promo
No_Promo    52.530196
DELIVERY    19.708235
DISCOUNT    14.105098
FREEBIE     13.656471
Name: count, dtype: float64


## numerical columns